In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from Pre_processing import *

In [3]:
df_curl = pd.read_csv(r'coords-curls.csv')
df_sqt = pd.read_csv(r'coords-squads.csv')

In [4]:
df_c = Pre_process(df_curl)
df_s = Pre_process(df_sqt)
df_c = df_c.replace('curls', 1)
df_s = df_s.replace('squats', 0)

In [5]:
df_total = pd.concat([df_c, df_s], axis=0)

In [6]:
df_total.head(2)

,vid.no,obs.no,frame.no,pose,y0,z0,y1,z1,y2,z2,y3,z3,y4,z4,y5,z5,y6,z6,y7,z7,y8,z8,y9,z9,y10,z10,y11,z11,y12,z12,y13,z13,y14,z14,y15,z15,y16,z16,y17,z17,y18,z18,y19,z19,y20,z20,y21,z21,y22,z22,y23,z23,y24,z24,y25,z25,y26,z26,y27,z27,y28,z28,y29,z29,y30,z30,y31,z31,y32,z32
0,1,1,1,1,0.018654,-0.81768,0.041363,-0.850829,0.054339,-0.848987,0.064071,-0.847145,0.000811,-0.848987,-0.012165,-0.847145,-0.02352,-0.845304,0.083536,-0.823204,-0.036496,-0.821363,0.047851,-0.775322,-0.007299,-0.773481,0.19708,-0.569061,-0.146796,-0.567219,0.190592,-0.257827,-0.187348,-0.252302,0.232766,0.055249,-0.239254,0.025783,0.273317,0.128913,-0.284672,0.095764,0.239254,0.130755,-0.265207,0.095764,0.218167,0.104972,-0.237632,0.073665,0.099757,0.007366,-0.099757,-0.007366,0.07056,0.366483,-0.161395,0.360958,0.049473,0.576427,-0.064071,0.515654,0.042985,0.604052,-0.038118,0.524862,0.060827,0.705341,-0.062449,0.640884
1,1,1,2,1,0.018654,-0.81768,0.041363,-0.850829,0.054339,-0.848987,0.064071,-0.847145,0.000811,-0.848987,-0.012165,-0.847145,-0.02352,-0.845304,0.083536,-0.823204,-0.036496,-0.821363,0.047851,-0.775322,-0.007299,-0.773481,0.19708,-0.569061,-0.146796,-0.567219,0.190592,-0.257827,-0.187348,-0.252302,0.232766,0.055249,-0.239254,0.025783,0.273317,0.128913,-0.284672,0.095764,0.239254,0.130755,-0.265207,0.095764,0.218167,0.104972,-0.237632,0.073665,0.099757,0.007366,-0.099757,-0.007366,0.07056,0.366483,-0.161395,0.360958,0.049473,0.576427,-0.064071,0.515654,0.042985,0.604052,-0.038118,0.524862,0.060827,0.705341,-0.062449,0.640884


In [7]:
train = df_total.iloc[:,4:].to_numpy()
Xtrain = np.reshape(train,(-1,50,66))

In [8]:
y1 = df_c.groupby(['vid.no','obs.no']).max()['pose']
y2 = df_s.groupby(['vid.no','obs.no']).max()['pose']

In [9]:
ytrain = pd.concat([y1, y2], axis=0).values

In [10]:
ytrain = np.reshape(ytrain,(-1,1))

In [28]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(Xtrain, ytrain, test_size = 0.25, random_state = 3)

In [29]:
x_train.shape

(365, 50, 66)

In [30]:
y_train.shape

(365, 1)

In [31]:
import torch
import torch.nn as nn
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [32]:
tensor_X_test = torch.from_numpy(x_test)
print('test_data_size:',tensor_X_test.size())
tensor_y_test = torch.from_numpy(y_test)
print('test_label_size:',tensor_y_test.size())
n_data_size_test = tensor_X_test.size()[0]
print('n_data_size_test:',n_data_size_test)

tensor_X_train = torch.from_numpy(x_train)
print('train_data_size:',tensor_X_train.size())
tensor_y_train = torch.from_numpy(y_train)
print('train_label_size:',tensor_y_train.size())
n_data_size_train = tensor_X_train.size()[0]
print('n_data_size_train:',n_data_size_train)

test_data_size: torch.Size([122, 50, 66])
test_label_size: torch.Size([122, 1])
n_data_size_test: 122
train_data_size: torch.Size([365, 50, 66])
train_label_size: torch.Size([365, 1])
n_data_size_train: 365


In [19]:
class LSTM(nn.Module):
    
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super(LSTM,self).__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim,hidden_dim,layer_num,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        self.bn = nn.BatchNorm1d(50)
        
    def forward(self,inputs):
        x = self.bn(inputs)
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [20]:
n_hidden = 128
n_joints = 33*2
n_categories = 2
n_layer = 3
rnn = LSTM(n_joints,n_hidden,n_categories,n_layer)
rnn.to(device)

LSTM(
  (lstm): LSTM(66, 128, num_layers=3, batch_first=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
  (bn): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [21]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return LABELS[category_i], category_i

In [22]:
LABELS = ['curls','squats']

In [23]:
import random
def randomTrainingExampleBatch(batch_size,flag,num=-1):
    if flag == 'train':
        X = tensor_X_train
        y = tensor_y_train
        data_size = n_data_size_train
    elif flag == 'test':
        X = tensor_X_test
        y = tensor_y_test
        data_size = n_data_size_test
    if num == -1:
        ran_num = random.randint(0,data_size-batch_size)
    else:
        ran_num = num
    pose_sequence_tensor = X[ran_num:(ran_num+batch_size)]
    pose_sequence_tensor = pose_sequence_tensor
    category_tensor = y[ran_num:ran_num+batch_size,:]
    return category_tensor.long(),pose_sequence_tensor

In [24]:
import torch.optim as optim
import time
import math

criterion = nn.CrossEntropyLoss()
learning_rate = 0.0005
optimizer = optim.SGD(rnn.parameters(),lr=learning_rate,momentum=0.9)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.1)

n_iters = 100000
#n_iters = 60000
print_every = 1000
plot_every = 1000
batch_size = 128

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
   
    category_tensor, input_sequence = randomTrainingExampleBatch(batch_size,'train')
    input_sequence = input_sequence.to(device)
    category_tensor = category_tensor.to(device)
    category_tensor = torch.squeeze(category_tensor)
    
    optimizer.zero_grad()
    
    output = rnn(input_sequence.float())
    loss = criterion(output, category_tensor)
    loss.backward()
    optimizer.step() 
    
    current_loss += loss.item()
    
    category = LABELS[int(category_tensor[0])]

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f  / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))
        
    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

1000 1% (0m 19s) 0.6864  / squats ✓
2000 2% (0m 39s) 0.6871  / curls ✗ (squats)
3000 3% (0m 59s) 0.6127  / squats ✓
4000 4% (1m 18s) 0.2627  / curls ✓
5000 5% (1m 38s) 0.0845  / squats ✓
6000 6% (1m 57s) 0.1268  / squats ✓
7000 7% (2m 17s) 0.0190  / curls ✓
8000 8% (2m 37s) 0.0073  / squats ✓
9000 9% (2m 56s) 0.0066  / curls ✓
10000 10% (3m 16s) 0.0599  / squats ✓
11000 11% (3m 35s) 0.0181  / curls ✓
12000 12% (3m 55s) 0.0035  / squats ✓
13000 13% (4m 14s) 0.0027  / curls ✓
14000 14% (4m 34s) 0.0016  / curls ✓
15000 15% (4m 54s) 0.0014  / squats ✓
16000 16% (5m 13s) 0.0011  / squats ✓
17000 17% (5m 32s) 0.0010  / squats ✓
18000 18% (5m 52s) 0.0010  / squats ✓
19000 19% (6m 12s) 0.0008  / curls ✓
20000 20% (6m 31s) 0.0007  / squats ✓
21000 21% (6m 51s) 0.0006  / squats ✓
22000 22% (7m 11s) 0.0006  / squats ✓
23000 23% (7m 30s) 0.0005  / curls ✓
24000 24% (7m 50s) 0.0005  / squats ✓
25000 25% (8m 9s) 0.0004  / squats ✓
26000 26% (8m 29s) 0.0004  / curls ✓
27000 27% (8m 48s) 0.0004  / squ

In [25]:
torch.save(rnn.state_dict(),'lstm_6_bn.pkl')

In [26]:
def test(flag):
    if flag == 'train':
        n = n_data_size_train
    elif flag == 'test':
        n = n_data_size_test   
        
    with torch.no_grad():
        right = 0
        for i in range(n):
            category_tensor, inputs = randomTrainingExampleBatch(1,flag,i)
            category = LABELS[int(category_tensor[0])]
            inputs = inputs.to(device)
            output = rnn(inputs)
            guess, guess_i = categoryFromOutput(output)
            category_i = LABELS.index(category)
            if category_i == guess_i:
                right+=1
    print(flag,'accuracy',right/n)